# Sparse Matrix Framework

Rather than looping through every cell and every timestep, we can instead just loop through every timestep, but solve the entire unstructured grid at once by setting up the grid as a sparse matrix. 

HEC-RAS 2D Sediment Transport solves an implicit Advection-Diffusion (transport) equation for the fractional total-load concentrations. The discretization produces a linear system of equations which may be represented by a sparse-matrix problem ([HEC-RAS 2D Sediment User Manual](https://www.hec.usace.army.mil/confluence/rasdocs/h2sd/ras2dsed/sediment-computation-options-and-tolerances/2d-computational-options)). The final form of the descritized total-load advection-diffusion equation can be seen [here](https://www.hec.usace.army.mil/confluence/rasdocs/d2sd/ras2dsedtr/numerical-methods/transport-equation). A similar approach can be taken for RAS2D-WQ. 

## Setup
Import packages and open results file. 

In [1]:
import xarray as xr 
import numpy as np
from scipy.sparse import *
from scipy.sparse.linalg import *

In [2]:
# Open Zarr
ds_zarr = xr.open_zarr('ugrid-example.zarr',
                       consolidated=True,  # http://xarray.pydata.org/en/stable/user-guide/io.html#consolidated-metadata
                      )

In [3]:
# additional required info - migrate to xarray setup code 
ds_zarr['volume'] = ds_zarr['depth'] * ds_zarr['faces_surface_area']
ds_zarr['edge_vertical_area'] = ds_zarr['depth'] * ds_zarr['edge_length']

In [4]:
ds_zarr

c:\Users\sjordan\Anaconda3\envs\pangeo\lib\site-packages\dask\array\core.py:1627: RuntimeWarning: overflow encountered in long_scalars
  cbytes = format_bytes(np.prod(self.chunksize) * self.dtype.itemsize)


<xarray.Dataset>
Dimensions:                 (time: 289, nedge: 11164, nface: 5765, 2: 2,
                             nmax_face: 7, node: 5774)
Coordinates:
    face_x                  (nface) float64 dask.array<chunksize=(5765,), meta=np.ndarray>
    face_y                  (nface) float64 dask.array<chunksize=(5765,), meta=np.ndarray>
    node_x                  (node) float64 dask.array<chunksize=(5774,), meta=np.ndarray>
    node_y                  (node) float64 dask.array<chunksize=(5774,), meta=np.ndarray>
  * time                    (time) datetime64[ns] 1900-01-02 ... 1900-01-03
Dimensions without coordinates: nedge, nface, 2, nmax_face, node
Data variables: (12/15)
    advection_coeff         (time, nedge) float64 dask.array<chunksize=(37, 2791), meta=np.ndarray>
    depth                   (time, nface) float32 dask.array<chunksize=(73, 1442), meta=np.ndarray>
    diffusion_coeff         (time, nedge) float64 dask.array<chunksize=(37, 2791), meta=np.ndarray>
    dt                      (time) timedelta64[ns] dask.array<chunksize=(289,), meta=np.ndarray>
    edge_face_connectivity  (nedge, 2) int32 dask.array<chunksize=(11164, 2), meta=np.ndarray>
    edge_length             (nedge) float32 dask.array<chunksize=(11164,), meta=np.ndarray>
    ...                      ...
    face_to_face_dist       (nedge) float64 dask.array<chunksize=(11164,), meta=np.ndarray>
    faces_surface_area      (nface) float32 dask.array<chunksize=(5765,), meta=np.ndarray>
    mesh2d                  int32 ...
    sum_diffusion_coeff     (time, nface) float64 dask.array<chunksize=(73, 1442), meta=np.ndarray>
    volume                  (time, nface) float32 dask.array<chunksize=(73, 1442), meta=np.ndarray>
    edge_vertical_area      (time, nface, nedge) float32 dask.array<chunksize=(73, 1442, 11164), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.8 UGRID-1.0 Deltares-0.10

In [5]:
# ultimately transition this to xarray attributes?
class Params:
    def __init__(self):
        self.diffusion_coefficient = 0.1
        self.beta = 1

params = Params() 
    

## Sparse Matrix Setup
A sparse matrix is a matrix that is mostly zeroes. Here, we will set up a `NCELL x NCELL` sparse matrix. The diagonal values represent the cell, and then the non-zero values in that row/column represent other cells that share an edge with that cell. The MODFLOW-USG theoretical documentation contains an excellent example of an unstructured grid and the corresponding sparse matrix that can help illustrate this description ([USGS, 2013](https://pubs.usgs.gov/tm/06/a45/pdf/tm6-A45.pdf)) -- see Figures 18 & 19. 

Useful links:
* [Cell Hydraulic Properties](https://www.hec.usace.army.mil/confluence/rasdocs/d2sd/ras2dsedtr/numerical-methods/cell-hydraulic-properties)
* [Face Hydraulic Properties](https://www.hec.usace.army.mil/confluence/rasdocs/d2sd/ras2dsedtr/numerical-methods/face-hydraulic-properties)

The class below is a work in progress for defining coefficients of the LHS matrix. Upcoming items to address:
* Do I need to sum on diagonals?
* Advection term:
    * [Upwind Differencing](https://www.youtube.com/watch?v=JVE0fNkc540)
* Leverage numpy operators  

In [6]:
class LHS:
    def __init__(self, ds, params):
        '''
        ds: xarray containing all geometry and ouptut results from RAS2D.
            Should follow UGRID conventions.
        params: A class instance containing additional parameters. 
            TBD if this will remain or if parameters will be integrated into xarray.
        t: timestep index  
        '''

        # initialize arrays that will define the sparse matrix 
        # rows, cols, data will all have a length of NCONNECTIONS * 2 + NCELL
        len_val = len(ds['nedge']) * 2 + len(ds['nface']) # update this - can be as long as we need
        # csr matrix will sum everything as long as we track indices: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

        self.rows = np.zeros(len_val)
        self.cols = np.zeros(len_val)
        self.coef = np.zeros(len_val)
        
    def updateValues(self, ds, params, t):
        # diagonal terms
        # TO DO: do I need to loop here or can I just use array properties?
        # Do not need to loop: need to update code. 
        for i in ds['nface']:
            # indices
            self.rows[i] = i
            self.cols[i] = i

            # diagonal coefficients
            volume = ds['volume'][t][i]
            self.coef[i] = volume/(ds['dt'][t] * params.beta) + ds['sum_diffusion_coeff'] # to update with information
        



        # off-diagonal terms
        # TO DO: streamline w/o loop necessary (numpy array)
        for i in range(len(ds['nedge'])):
            # indices of connected faces for each edge 
            face_index_1 = ds['edge_face_connectivity'][i][0]
            face_index_2 = ds['edge_face_connectivity'][i][1]

            ### KEEP WORKING ON THIS.
            rindex = face_index_1[i]
            if ds['advection_coeff'][i] < 0:
                # into cell: on the diagonal
                cindex = face_index_1[i]
            else:
                # out of cell: off diagonal
                cindex = face_index_2[i]


            index_val = len(ds['nface']) + 2*i + 1
            
            self.rows[index_val] = face_index_1
            self.cols[index_val] = face_index_2
            self.coef[index_val] = -1 * ds['diffusion_coeff'][t][i]  # need to add advection
            
            self.rows[index_val + 1] = face_index_2
            self.cols[index_val + 1] = face_index_1
            self.coef[index_val + 1] = -1 * ds['diffusion_coeff'][t][i] # need to add advection





To do:
* Recompute the length of the row/column/data matrices based on above plus the number of cells relevant for advection (length of flow in indices and length of flow out indices)
* Use the scheme below to add data to the diagonal / off-diagonal depending on data
* Figure out the signs

In [44]:
# to go in LHS
# advection
t = 200 # test at a given timestep 
flow_out_indices = np.where(ds_zarr['advection_coeff'][t] < 0)
flow_in_indices = np.where(ds_zarr['advection_coeff'][t] > 0)

# where advection coefficient is positive, the concentration across the face will be the REFERENCE CELL 
# so the the coefficient will go in the diagonal - both row and column will equal diag_cell
# CHECK SIGNS! 
diag_cells = ds_zarr['edge_face_connectivity'].T[0][flow_in_indices]
diag_data = ds_zarr['advection_coeff'][t][flow_in_indices]

# where it is negative, the conecentration across the face will be the neighbor cell
# so the coefficient will be off diagonal 
row = ds_zarr['edge_face_connectivity'].T[0][flow_out_indices]
col = ds_zarr['edge_face_connectivity'].T[1][flow_out_indices]
data = ds_zarr['advection_coeff'][t][flow_out_indices]
# will need to add this twice maybe?


In [46]:
diag_data.compute()

<xarray.DataArray 'advection_coeff' (nedge: 2465)>
array([1.95575635e+03, 1.22043381e+02, 1.94318750e+03, ...,
       1.34533376e-01, 6.84601545e-01, 3.12871674e+02])
Coordinates:
    time     datetime64[ns] 1900-01-02T16:40:00
Dimensions without coordinates: nedge
Attributes:
    units:    cubic feet

In [47]:
data.compute()

<xarray.DataArray 'advection_coeff' (nedge: 7340)>
array([-1.18768356e+02, -1.74555206e+02, -1.74124329e+02, ...,
       -1.07531935e-01, -7.29059696e-01, -8.16208839e-01])
Coordinates:
    time     datetime64[ns] 1900-01-02T16:40:00
Dimensions without coordinates: nedge
Attributes:
    units:    cubic feet